In [2]:
import pkg_resources
# was running numpy version 1.19.2 which isn't compatible w/ tensorflow
# force to run updated version of numpy
pkg_resources.require("numpy==1.22.2")  
import numpy as np

# libraries for data cleaning and preparation for model
import os
import os.path
import chess
import chess.pgn

# libraries for GAN model
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint

from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D,Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.initializers import RandomNormal
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Activation,Reshape
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dropout
from IPython.display import clear_output
from keras.models import load_model


# libraries for visuals
import matplotlib.pyplot as mpl
import pydot
import pydotplus
import graphviz

In [3]:
def extractdata(pgn_path):
    '''
    input: pgn files of player
    output: 
        function returns step-by-step gameplay as a list
        function returns which player is playing White as a list
    
    list 'side' will be used to ensure only moves made by 
    intended player will be used in creation of the GAN
    '''
    pgn = open(pgn_path, encoding='utf-8')
    
    side = []
    game_moves = []
    length = 2 #used for training purposes, to remove for DA servers
    for index in range(length):
        try:
            this_game = chess.pgn.read_game(pgn)
            game_moves.append(this_game.mainline_moves()) 
            side.append(this_game.headers["White"]) 
        
        except:
            break
                

    return game_moves, side

In [4]:
def categorize_moves(game_moves, side, name):
    '''
    input: game_moves and side list from extractdata function
    output:
        function returns 2 lists, which contain all of player's move
        list PW: player's moves when they are playing white
        list PB: player's moves when they are playing black 
    '''
    
    PW = [] # empty list for all moves when player playing white
    PB = [] # empty list for all moves when player playing black
    
    match = 0
    
    for game in game_moves:
        board = chess.Board() # saves FEN notation of chess board
        white = side[match]

        count = 0
        if name in white:
            index = 1
            for move in game:
                if index % 2 == 0:
                    PW.append(board.copy()) # creates list PW of moves when the player is playing white
                board.push(move) # move game forward one move
                index = index + 1
                
    
        if name not in white:
            index = 0
            for move in game:
                board.push(move)
                
                if index % 2 == 1:
                    PB.append(board.copy()) # creates list PB of moves when the player is playing black
                index = index + 1
        
        match = match + 1
       
    return PW, PB

In [5]:
def make_matrix(board): 
    '''
    input: FEN notation of a board position
    output: matrix representing board position at a given moment
        each board square is an individual item in the matrix, blank squares are represented by a period
    '''
    
    pgn = board.epd() # convert FEN notation of board into EPD notation
    matrix = []  

    # retrieve only the first field from EPD notation: the piece placement
    pieces = pgn.split(" ", 1)[0] 
    
    # separate into placement of individual pieces
    rows = pieces.split("/")
    
    # separates rows so that each specific square on the board is its own list entry, formatted as a matrix
    for row in rows:
        game_row = []  
        for item in row:
            if item.isdigit():
            
            # replaces numbers in epd that represent the number of empty squares with a period for each empty square
            # example: '8' = '........'
            
                for i in range(0, int(item)):
                    game_row.append('.')
            else:
                game_row.append(item)
        matrix.append(game_row)
    return matrix

In [6]:
# translate game pieces to binary values using one-hot encoding
# each game piece is represented by a unique binary vector

chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1,0],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0,1],
}

In [7]:
def translate(matrix, chess_dict):
    '''
    input: matrix created in the previous function, chess dictionary
    output: layout of a chess board represented by a matrix of one-hot encoded values
    '''
    
    rows = []
    for row in matrix:
        pieces = []
        for piece in row:

            # appends one-hot endoded value associated with each chess piece, pulled from chess_dict
            pieces.append(chess_dict[piece])
        rows.append(pieces)
    return rows

In [8]:
def one_hot_matrix(X, Y):
    '''
    inputs:
        X: list of moves when player is playing white
        Y: list of moves when player is playing black
        
    translate and make_matrix functions to convert each instance of the game board into a one-hot encoded matrix
    then transforms matrix into numpy array
    
    outputs:
        X_array: numpy array of all moves when player is playing white
        Y_array: numpy array of all moves when player is playing black
    '''
    for i in range(len(X)):
        X[i] = translate(make_matrix(X[i]),chess_dict)
    for i in range(len(Y)):
        Y[i] = translate(make_matrix(Y[i]),chess_dict)
    X_array = np.array(X)
    Y_array = np.array(Y)

    return X_array, Y_array

In [9]:
def define_discriminator(image_shape = (8,8,13)):
    '''
    create the PatchGAN discriminator model
    
    model takes a chess move from the source domain (real move) and a chess move from the target domain 
    (imitated move) and predicts the likelihood (known as the error) of whether the move from the target 
    domain is a real or generated version of the chess player's gameplay
    '''
    
    # initiatlize as randomly distributed with a SD of 0.02
    init = RandomNormal(stddev=0.02)
    
    # real chess move input
    in_src_image = Input(shape=image_shape)
    
    # imitated chess move input
    in_target_image = Input(shape=image_shape)

    # concatenate images channel-wise
    # create one 256x256x6 input to the first hidden convolutional layer
    merged = concatenate([in_src_image, in_target_image])
    
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    
    #normalizes output using moving average of mean and standard deviation of the batches it has seen during training.
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # second to last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    
    # define model
    model = Model(inputs = [in_src_image, in_target_image], outputs = patch_out)

    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [10]:
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    '''
    this function defines the encoder block of the final generator function
    
    inputs: 
        current layer of model (layer_in), number of filters (n_filters), presence or absence of batch normalization
        
    output: encoded layer
    '''
    # initiatlize as randomly distributed with a SD of 0.02
    init = RandomNormal(stddev=0.02)
   
    # add downsampling layer
    # flattes given image to a 2D structure
    # convolutional layers that use a 2×2 stride to downsample the input source image down to a bottleneck layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    
    # conditionally add batch normalization
    # normalizes its output using the mean and standard deviation of the current batch of inputs
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    
    # leaky relu activation
    # more balanced than regular relu, may learn faster
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [11]:
def define_decoder_block(layer_in, skip_in, n_filters, dropout=True):
    '''
    this function defines the decoder block of the final generator function
    uses transpose convolutional layers to upsample to required output image size
    
    inputs: 
        current layer of model, encoded (layer_in), 
        skip connection input (skip_in), number of filters (n_filters), 
        introduction of dropout layers as a source of randomness
    
    output: decoded layer
    '''

    # initiatlize as randomly distributed with a SD of 0.02
    init = RandomNormal(stddev=0.02)
    
    # add unsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    
    # add batch normalization
    # normalizes its output using the mean and standard deviation of the current batch of inputs
    g = BatchNormalization()(g, training=True)
    
    # conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
    
    # merge with skip connection
    g = concatenate([g, skip_in])
    # relu activation
    g = Activation('relu')(g)
    return g

In [12]:
def define_generator(image_shape=(8,8,13)):
    '''
    function defines standalone generator model

    inputs: image_shape, for chess board it is 8 x 8 x 13
        8 x 8 represents size of the board, 13 represents depth, or number of possible pieces that could
        fill each chess board square, as defined above in chess_dict
        
    output: standalone generator model that creates chess gameplay moves that attempt to imitate the player
    and 'trick' the discriminator into believing it is a real move by that player
    '''

    # initiatlize as randomly distributed with a SD of 0.02
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)

    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e2)
    b = Activation('relu')(b)
    
    # decoder model
    d6 = define_decoder_block(b, e2, 128, dropout=False)
    d7 = define_decoder_block(d6, e1, 64, dropout=False)
    g = Conv2DTranspose(13, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)

    # 'softmax' converts a vector of values to a probability distribution
    out_image = Activation('softmax')(g)

    # define generator model itself
    model = Model(in_image, out_image)
    return model

In [13]:
def define_gan(g_model, d_model, image_shape):
    '''
    function defined the final GAN model
    inputs: generator model, discriminator model, shape of image to be generated 
        shape will be 8x8x13 to represent possible positions on a chess board
        
    output: usable GAN model
    '''
    d_model.trainable = False
    # define source image
    in_src = Input(shape=image_shape)
    # connect source input to the generator input
    gen_out = g_model(in_src)
    # connect source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    
    # source image as input, generated image and classification (real/fake) as output
    model = Model(in_src, [dis_out, gen_out])

    # compile final model with Adam optimizer
    # Adam optimizer:
        # computationally efficient, doesn't require a lot of memory, maintains single learning rate
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [14]:
def generate_real_samples(dataset, n_samples, patch_shape):
    '''
    function pulls real samples of chess moves from the dataset
    
    inputs: pgn file of players moves (dataset), number of samples to create (n_samples),
    shape of discriminator model output (patch_shape)
    
    outputs: real chess move data
    '''
    trainA, trainB = dataset
    # choose random chess moves from dataset
    ix_x = randint(0, trainA.shape[0], n_samples)
    ix_y = randint(0, trainB.shape[0], n_samples)
    
    # selects lower of two random indices to ensure that each dataset has an observation at that point
    if ix_x <= ix_y:
        ix = ix_x
    else:
        ix = ix_y
    
    X1, X2 = trainA[ix], trainB[ix]
    # labels them with '1' to designate that they are real
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y

In [15]:
def generate_fake_samples(g_model, samples, patch_shape):
    '''
    function creates fake samples of chess moves using the generator model
    
    inputs: generator model (g_model), real samples from dataset (samples), 
    and shape of discriminator model output (patch_shape)
    
    outputs: fake samples of chess moves
    '''
    # generate fake instance
    X = g_model.predict(samples)
    # labels them with '0' to designate that they are real
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [16]:
def train(d_model, g_model, gan_model, dataset, n_epochs=1, n_batch=1):
    '''
    function trains the model to imitate the chess player
    
    inputs: disrimiantor model, generator model, combined GAN model, data to be used
    '''
    n_patch = d_model.output_shape[1]
    trainA, trainB = dataset
    bat_per_epo = int(len(trainA) / n_batch)
    n_steps = bat_per_epo * n_epochs
    d_loss1_list = []
    d_loss2_list = []
    g_loss_list = []
    steps = []
    
    for i in range(n_steps):
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        d_loss1_list.append(d_loss1)
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        d_loss2_list.append(d_loss2)
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        g_loss_list.append(g_loss)
        steps.append(i)        
        # use during training to see step-by-step loss values
        #print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    if (i+1) % (bat_per_epo * 10) == 0:
        clear_output()
    return d_loss1_list, d_loss2_list, g_loss_list, steps, gan_model

In [17]:
alekhine = "/Users/zoepratt/Documents/GitHub/Top-Chess-Players/code/alekhine.pgn"
carlsen = "/Users/zoepratt/Documents/GitHub/Top-Chess-Players/code/carlsen.pgn"

**Carlsen GAN**

In [18]:
game_moves, side = extractdata(carlsen)
PW, PB = categorize_moves(game_moves, side, 'Carlsen')
X, y = one_hot_matrix(PW, PB)
d_model = define_discriminator(image_shape = (8,8,13))
g_model = define_generator()
gan_model = define_gan(g_model, d_model, image_shape = (8,8,13))
d_loss1_list, d_loss2_list, g_loss_list, steps, gan_model = train(d_model, g_model, gan_model, [X, y])
gan_model.save('carlsen_test.h5') # save model to computer file

**Alekhine GAN**

In [19]:
game_moves, side = extractdata(alekhine)
PW, PB = categorize_moves(game_moves, side, 'Alekhine')
X, y = one_hot_matrix(PW, PB)
d_model = define_discriminator(image_shape = (8,8,13))
g_model = define_generator()
gan_model = define_gan(g_model, d_model, image_shape = (8,8,13))
d_loss1_list, d_loss2_list, g_loss_list, steps, gan_model = train(d_model, g_model, gan_model, [X, y])
gan_model.save('alekhine_test.h5') # save model to computer

In [20]:
carlsen_file = "/Users/zoepratt/Documents/GitHub/Top-Chess-Players/code/carlsen_test.h5"
carlsen = load_model(carlsen_file)

alekhine_file = "/Users/zoepratt/Documents/GitHub/Top-Chess-Players/code/alekhine_test.h5"
alekhine = load_model(alekhine_file)

In [72]:
new_chess_dict = {
    (1,0,0,0,0,0,0,0,0,0,0,0,0) : 'p',
    (0,0,0,0,0,0,1,0,0,0,0,0,0) : 'P',
    (0,1,0,0,0,0,0,0,0,0,0,0,0) : 'n',
    (0,0,0,0,0,0,0,1,0,0,0,0,0) : 'N',
    (0,0,1,0,0,0,0,0,0,0,0,0,0) : 'b',
    (0,0,0,0,0,0,0,0,1,0,0,0,0) : 'B',
    (0,0,0,1,0,0,0,0,0,0,0,0,0) : 'r',
    (0,0,0,0,0,0,0,0,0,1,0,0,0) : 'R',
    (0,0,0,0,1,0,0,0,0,0,0,0,0) : 'q',
    (0,0,0,0,0,0,0,0,0,0,1,0,0) : 'Q',
    (0,0,0,0,0,1,0,0,0,0,0,0,0) : 'k',
    (0,0,0,0,0,0,0,0,0,0,0,1,0) : 'K',
    (0,0,0,0,0,0,0,0,0,0,0,0,1) : '.'
}

In [195]:
polgar_file = "/Users/zoepratt/Documents/GitHub/Top-Chess-Players/code/polgar.h5"
polgar = load_model(polgar_file)

In [248]:
# starting board position
chess.STARTING_BOARD_FEN= 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR'
board = chess.Board()

# matrix of starting board, for predicting
matrix = translate(make_matrix(board),chess_dict)
array = np.array(matrix)
array = array.reshape(1,8,8,13)

In [251]:
import random
flatten = lambda l: [item for sublist in l for item in sublist]

instance = random.randint(1, len(X)-1)
state = X[instance].reshape(1,8,8,13)

def draw_state(state):
    pre_board = []
    for move in range(len(state)):
        for i in state[move]:
            for j in i:
                pre_board.append(j)
    pre_board = list(pre_board)

    for i in range(len(pre_board)):
        pre_board[i] = new_chess_dict[tuple(pre_board[i])]
    pre_board = np.array(pre_board).reshape(8,8)
    return  pre_board

print(draw_state(array))

[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['P' 'P' 'P' 'P' 'P' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]


In [255]:
action = polgar.predict(array)[1]
board = []
flatten_action = flatten(flatten(action))
print(flatten_action)
for i in range(len(flatten_action)):
    new_set = np.zeros((13,))
    max_index = list(flatten_action[i]).index(max(flatten_action[i]))
    new_set[max_index] = 1
    board.append(new_set)
for i in range(len(board)):
    #print(board[i])
    board[i] = new_chess_dict[tuple(board[i])]
board = np.array(board).reshape(8,8)
print(board)


[array([2.1692547e-11, 1.4394043e-11, 1.8468079e-11, 1.7580217e-09,
       1.5583810e-11, 2.0196437e-11, 4.5335586e-11, 1.2098710e-11,
       1.8731631e-11, 1.4072625e-11, 1.3437929e-11, 1.0268571e-11,
       1.0000000e+00], dtype=float32), array([1.9433328e-14, 2.4502058e-14, 3.3400142e-14, 4.5564457e-14,
       4.2195488e-14, 1.0579691e-14, 1.8046634e-11, 9.3937767e-15,
       9.9947423e-15, 1.6824912e-14, 1.1836360e-14, 1.8101385e-14,
       1.0000000e+00], dtype=float32), array([1.8447351e-14, 6.9687827e-15, 7.8240425e-15, 6.2586343e-14,
       4.5981666e-15, 2.3746941e-14, 5.2915517e-14, 8.1098433e-15,
       6.9725054e-15, 2.9688448e-15, 5.3206146e-15, 2.6247478e-15,
       1.0000000e+00], dtype=float32), array([2.9838331e-14, 2.9935412e-14, 1.8135494e-14, 3.7169412e-14,
       3.3187029e-14, 9.3355838e-15, 5.8926204e-11, 1.8353994e-14,
       1.5936557e-14, 2.1954875e-14, 1.4757443e-14, 1.7334905e-14,
       1.0000000e+00], dtype=float32), array([1.9978859e-14, 7.4439695e-15, 1.